# Lista 12 - Classificação de Imagens com CNNs (Gatos vs Cachorros)
Disciplina: Inteligência Artificial  
Professora: Cristiane Neri Nobre

In [ ]:
# Etapa 1: Importação das bibliotecas
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import tensorflow as tf

## Etapa 2: Preparação dos dados

In [ ]:
img_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_val_datagen.flow_from_directory(
    'dataset/validation',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_val_datagen.flow_from_directory(
    'dataset/test',
    target_size=img_size,
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

## Etapa 3: Construção e treinamento da CNN

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

## Etapa 4: Avaliação

In [ ]:
test_generator.reset()
pred = model.predict(test_generator)
pred_labels = (pred > 0.5).astype(int)
true_labels = test_generator.classes
print(classification_report(true_labels, pred_labels, target_names=["Cat", "Dog"]))

## Etapa 5: Gráficos de desempenho

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treinamento')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia por época')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treinamento')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda por época')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.show()

## Etapa 6: Teste com novas imagens

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = 'imagens_novas/gato1.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img) / 255.
img_tensor = np.expand_dims(img_tensor, axis=0)

prediction = model.predict(img_tensor)
print("Predição:", "Cachorro" if prediction[0] > 0.5 else "Gato")

plt.imshow(img)
plt.title("Cachorro" if prediction[0] > 0.5 else "Gato")
plt.axis('off')
plt.show()